#### QUESTIONS
1. What is padding, output_padding in nn.ConvTranspose2d?
2. What is ReflectionPad2d v.s. ReplicationPad2d?

In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from fastai.vision import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def conv_t_norm_relu(ch_in, ch_out, norm_layer, ks = 3, stride = 2, bias = True):
    conv_t = nn.ConvTranspose2d(ch_in, ch_out, kernel_size = ks, stride = stride, 
                                padding = 1, output_padding = 1, bias = bias)
    return [conv_t, norm_layer(ch_out), nn.ReLU(True)]

In [3]:
def pad_conv_norm_relu(ch_in, ch_out, pad_mode, norm_layer, ks = 3, bias = True,
                        pad = 1, stride = 1, activ = True, init = nn.init.kaiming_normal_):
    layers = []
    if pad_mode == 'reflection':
        layers.append(nn.ReflectionPad2d(pad))
    elif pad_mode == 'border':
        layers.append(nn.ReplicationPad2d(pad))
    p = pad if pad_mode == 'zeros' else 0
    conv = nn.Conv2d(ch_in, ch_out, kernel_size = 3, padding = p, stride = stride, bias = bias)
    if init:
        init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'): conv.bias.data.fill_(0.)
    layers += [conv, norm_layer(ch_out)]
    if activ: layers.append(nn.ReLU(True))
    return layers

In [4]:
pad_conv_norm_relu(1, 3, 'reflection', nn.InstanceNorm2d)

[ReflectionPad2d((1, 1, 1, 1)),
 Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1)),
 InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False),
 ReLU(inplace=True)]